In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from collections import Counter
from datetime import datetime

In [ ]:
award = pd.read_csv('/content/drive/MyDrive/SI670Final/awards_data.csv')
player = pd.read_csv('/content/drive/MyDrive/SI670Final/player_stats.csv')
team_rebound = pd.read_csv('/content/drive/MyDrive/SI670Final/team_rebounding_data_22.csv')
team_stat = pd.read_csv('/content/drive/MyDrive/SI670Final/team_stats.csv')

### Find players who are all star

In [ ]:
all_star_players = award[award['all_star_game']==True]

In [ ]:
 # merge name to all star player data
 player_all_star = pd.merge(all_star_players,player[['player','nbapersonid']],on='nbapersonid', how='left')

In [ ]:
# choose players that are drafted btw 2007, 2017
player07=player[player['draftyear'].between(2007,2017)]

In [ ]:
player07.player.value_counts()

Ish Smith       18
Corey Brewer    18
Jeff Green      18
Greg Monroe     17
Goran Dragic    17
                ..
Tibor Pleiss     1
J.J. O'Brien     1
Mike Tobey       1
A.J. Hammons     1
Craig Sword      1
Name: player, Length: 883, dtype: int64

### Make is_all_star column as 1, 0

In [ ]:
player07['is_all_star'] = player07['player'].apply(lambda name: 1 if name in set(player_all_star['player']) else 0)


<ipython-input-8-f281ff975980>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player07['is_all_star'] = player07['player'].apply(lambda name: 1 if name in set(player_all_star['player']) else 0)


### Make season as the first to fourth year

In [ ]:
player07_filtered2 = player07[player07['season'] - player07['draftyear'] <=3]

In [ ]:
player07_filtered2['Year']=player07_filtered2['season']-player07_filtered2['draftyear']+1

<ipython-input-10-08efab7dd12b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player07_filtered2['Year']=player07_filtered2['season']-player07_filtered2['draftyear']+1


### remove useless column

In [ ]:
df=player07_filtered2.drop(['player', 'draftyear', 'season', 'nbateamid', 'team'],axis=1)

### Impute missing value

In [ ]:
df.isna().sum()

nbapersonid      0
draftpick      576
games            0
games_start      0
mins             0
fgm              0
fga              0
fgp             20
fgm3             0
fga3             0
fgp3           360
fgm2             0
fga2             0
fgp2            45
efg             20
ftm              0
fta              0
ftp            185
off_reb          0
def_reb          0
tot_reb          0
ast              0
steals           0
blocks           0
tov              0
tot_fouls        0
points           0
PER              0
FTr             20
off_reb_pct      0
def_reb_pct      0
tot_reb_pct      0
ast_pct          0
stl_pct          0
blk_pct          0
tov_pct         15
usg              0
OWS              0
DWS              0
WS               0
OBPM             0
DBPM             0
BPM              0
VORP             0
is_all_star      0
Year             0
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='constant', fill_value=100)

In [ ]:
### impute 100 for the undrafted players
df['draftpick'] = imputer.fit_transform(df[['draftpick']])

In [ ]:
df.isna().sum()

nbapersonid      0
draftpick        0
games            0
games_start      0
mins             0
fgm              0
fga              0
fgp             20
fgm3             0
fga3             0
fgp3           360
fgm2             0
fga2             0
fgp2            45
efg             20
ftm              0
fta              0
ftp            185
off_reb          0
def_reb          0
tot_reb          0
ast              0
steals           0
blocks           0
tov              0
tot_fouls        0
points           0
PER              0
FTr             20
off_reb_pct      0
def_reb_pct      0
tot_reb_pct      0
ast_pct          0
stl_pct          0
blk_pct          0
tov_pct         15
usg              0
OWS              0
DWS              0
WS               0
OBPM             0
DBPM             0
BPM              0
VORP             0
is_all_star      0
Year             0
dtype: int64

In [ ]:
### fill zero for all other missing vlaue because they are 0
df.fillna(0)

,nbapersonid,draftpick,games,games_start,mins,fgm,fga,fgp,fgm3,fga3,...,usg,OWS,DWS,WS,OBPM,DBPM,BPM,VORP,is_all_star,Year
3,201151,11.0,56,6,865,95,237,0.401,7,34,...,0.165,-0.5,0.4,-0.1,-4.2,-1.0,-5.2,-0.7,0,1
10,201143,3.0,81,77,2540,333,668,0.499,0,5,...,0.160,2.4,3.0,5.5,-1.2,-0.3,-1.6,0.3,1,1
15,201238,100.0,64,2,269,21,49,0.429,0,4,...,0.117,0.2,0.2,0.5,-4.4,0.6,-3.8,-0.1,0,1
19,201157,17.0,73,29,1278,164,305,0.538,0,0,...,0.155,0.8,1.7,2.5,-3.3,1.1,-2.2,-0.1,0,1
47,201172,32.0,15,0,95,14,39,0.359,3,12,...,0.220,-0.1,0.2,0.1,-5.7,0.8,-4.8,-0.1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,1628378,13.0,53,53,1771,478,1091,0.438,178,461,...,0.335,3.9,2.2,6.2,4.1,-0.6,3.5,2.5,1,4
7749,1629744,100.0,19,0,134,26,65,0.400,10,39,...,0.245,-0.1,0.2,0.1,-3.9,-2.0,-6.0,-0.1,0,4
7762,1628395,38.0,5,1,67,7,20,0.350,0,2,...,0.142,-0.1,0.1,0.0,-4.7,-0.8,-5.5,-0.1,0,4
7765,1628418,42.0,10,10,271,59,91,0.648,9,21,...,0.172,0.7,0.2,1.0,2.1,-1.7,0.4,0.2,0,4


In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
import pandas as pd

# Assuming player07_filtered2 is your DataFrame

# Define the columns for which you want to perform the aggregation
columns_to_aggregate = ['fgp3', 'fgp2', 'efg', 'ftp', 'PER', 'FTr', 'off_reb_pct', 'def_reb_pct', 'tot_reb_pct', 'ast_pct', 'stl_pct',
                        'blk_pct', 'tov_pct', 'usg', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP' ]  # Add all your columns here

# Define custom aggregation function
def weighted_average(x, weight_column):
    return sum(x * df.loc[x.index, weight_column]) / sum(df.loc[x.index, weight_column])

# Create a dictionary of aggregations
aggregations = {}
for column in columns_to_aggregate:
    if column == 'games':
        aggregations[column] = 'sum'
    else:
        aggregations[column] = lambda x: weighted_average(x, 'games')

# Add columns to sum
columns_to_sum = ['games','games_start','mins','fgm','fga','fgm3','fga3','fgm2','fga2','ftm',
                  'fta','off_reb','def_reb','tot_reb','ast','steals','blocks','tov','tot_fouls','points']  # Add columns to sum

aggregations.update({column: 'sum' for column in columns_to_sum})

# Group by player and season, applying custom aggregation
df_aggregated = df.groupby(['nbapersonid', 'Year']).agg(aggregations).reset_index()

# Rename the columns for clarity if needed
# player07_aggregated.columns = [f'{column}_aggregated' for column in player07_aggregated.columns]

# Display the aggregated DataFrame
print(df_aggregated)


      nbapersonid  Year      fgp3      fgp2       efg    ftp        PER  \
0          201141     2       NaN  0.564000  0.564000  0.637  18.100000   
1          201141     3       NaN  0.605000  0.605000  0.766  23.100000   
2          201142     1  0.288000  0.455000  0.451000  0.873  15.800000   
3          201142     2  0.422000  0.486000  0.510000  0.863  20.800000   
4          201142     3  0.365000  0.506000  0.514000  0.900  26.200000   
...           ...   ...       ...       ...       ...    ...        ...   
2284      1629353     2  0.273000  0.300000  0.357000    NaN   3.400000   
2285      1629741     3  0.301000  0.500000  0.475000  0.923   8.800000   
2286      1629744     3  0.475000  0.510000  0.643000  0.750  13.300000   
2287      1629744     4  0.347867  0.452067  0.504156  0.857   9.171111   
2288      1630466     4  0.133000  0.577000  0.493000  0.818  16.100000   

           FTr  off_reb_pct  def_reb_pct  ...  fta  off_reb  def_reb  tot_reb  \
0     0.644000    

In [ ]:
# Pivot the DataFrame
pivoted_df = df.pivot_table(index='nbapersonid', columns='Year', values=df.columns[2:], fill_value=0)

# Flatten the MultiIndex columns
pivoted_df.columns = [f'{col[0]}_{col[1]}' for col in pivoted_df.columns]

# Reset the index to make 'nbapersonid' a regular column
pivoted_df.reset_index(inplace=True)

# Print or use the resulting DataFrame 'pivoted_df'
print(pivoted_df)

     nbapersonid  BPM_1  BPM_2  BPM_3  BPM_4  DBPM_1  DBPM_2  DBPM_3  DBPM_4  \
0         201141    0.0   -1.1    3.5   0.00     0.0     0.3     2.6    0.00   
1         201142   -0.1    3.3    7.1   4.90    -0.7    -0.1     0.8   -0.30   
2         201143   -1.6    1.7    2.3   3.60    -0.3     1.5     0.5    1.10   
3         201144   -3.0   -0.3   -0.6   1.20    -1.3    -0.3    -0.6    0.50   
4         201145   -4.1   -1.2    0.2  -0.95    -0.6    -0.6     0.6   -0.05   
..           ...    ...    ...    ...    ...     ...     ...     ...     ...   
800      1629152    0.0    0.0   -5.8   0.00     0.0     0.0     2.1    0.00   
801      1629353    0.0  -11.0    0.0   0.00     0.0    -4.7     0.0    0.00   
802      1629741    0.0    0.0   -3.4   0.00     0.0     0.0    -0.7    0.00   
803      1629744    0.0    0.0    1.1  -5.35     0.0     0.0     0.1   -2.25   
804      1630466    0.0    0.0    0.0  -0.40     0.0     0.0     0.0   -0.10   

     DWS_1  ...  tov_3  tov_4  tov_pct_

In [ ]:
### merge true lable to the dataset
player07_filtered3 = player07_filtered2[['nbapersonid', 'is_all_star']]
player07_filtered3.drop_duplicates(inplace = True)
pivoted_df.drop(columns = ['is_all_star_1', 'is_all_star_2', 'is_all_star_3', 'is_all_star_4',
                 'draftpick_1', 'draftpick_2', 'draftpick_3', 'draftpick_4'], inplace = True)

<ipython-input-20-b07ae7a947c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player07_filtered3.drop_duplicates(inplace = True)


In [ ]:
df['draftpick'].isnull().sum()

0

In [ ]:
df_dp = df[['draftpick', 'nbapersonid']].drop_duplicates()

In [ ]:
df_dp

,draftpick,nbapersonid
0,11.0,201151
1,3.0,201143
2,100.0,201238
3,17.0,201157
4,32.0,201172
...,...,...
2533,55.0,1628430
2536,25.0,1628394
2562,100.0,1628419
2566,100.0,1628578


In [ ]:
pivoted_df2 = pd.merge(pivoted_df, player07_filtered3[['is_all_star','nbapersonid']],how = 'left', on = 'nbapersonid')

In [ ]:
pivoted_df2 = pd.merge(pivoted_df2, df_dp,how='left',on='nbapersonid')

In [ ]:
pivoted_df2.isnull().sum()

nbapersonid    0
BPM_1          0
BPM_2          0
BPM_3          0
BPM_4          0
              ..
usg_2          0
usg_3          0
usg_4          0
is_all_star    0
draftpick      0
Length: 171, dtype: int64

In [ ]:
pivoted_df2.to_csv('pivoted_data_with_y_new.csv')